## Environment Setup

In [ ]:
%pip install pandas
%pip install plotly-express
%pip install --upgrade nbformat
%pip install ipykernel
%pip install -U kaleido

In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np

## Dataset Loading

In [ ]:
fact_map_point_state = pd.read_csv("training_data/fact_map_point_state.tsv", sep="\t", index_col=["time", "map_point_id"])
fact_map_point_state.head()

In [ ]:
fact_step = pd.read_csv("training_data/fact_step.tsv", sep="\t", index_col=["start_time", "robot_id"])
fact_step.head()

In [ ]:
fact_memory_map_point = pd.read_csv("training_data/fact_memory_map_point.tsv", sep="\t", index_col=["time", "robot_id", "x", "y"])
fact_memory_map_point.head()

In [ ]:
dym_map_point = pd.read_csv("training_data/dym_map_point.tsv", sep="\t", index_col=["id"])
dym_map_point.head()

In [ ]:
dym_robot = pd.read_csv("training_data/dym_robot.tsv", sep="\t", index_col=["id"])
dym_robot.head()

## Map Visualizations

In [ ]:
terrain_colors = {
    'Free': 'white',
    'Fire': 'red',
    'Base': 'blue',
    'OccupiedByRobot': 'gray',
    'Obstacle': 'brown',
    'OccupiedByItem': 'green',
}

In [ ]:
ROBOT_ID = 10

idx = pd.IndexSlice
robot_steps = fact_memory_map_point.loc[idx[:, ROBOT_ID, :, :]]

fig = px.scatter(robot_steps,
    x=robot_steps.index.get_level_values("x"), 
    y=robot_steps.index.get_level_values("y"), 
    color='terrain_type', 
    animation_frame=robot_steps.index.get_level_values("time"),
    title='Robot Map Memory Over Time',
    labels={'x': 'X Coordinate', 'y': 'Y Coordinate'},
    size_max=10,
    color_discrete_map=terrain_colors,
)
fig.update_layout(
    xaxis=dict(range=[0, 50], title="X Coordinate", showgrid=False, zeroline=False),
    yaxis=dict(range=[0, 50], title="Y Coordinate", showgrid=False, zeroline=False),
    legend_title="Terrain Type",
    width=600,
    height=600,
    title_x=0.5,
    plot_bgcolor="black",
)

fig.show()

## Interaction Processing

In [ ]:
def compare_orientation(line):
    if line["measured_orientation"] == line["real_orientation"]:
        return 0
    if ((line["measured_orientation"] in ["Up", "Down"]
        and line["real_orientation"] in ["Up", "Down"])
        or (line["measured_orientation"] in ["Left", "Right"]
        and line["real_orientation"] in ["Left", "Right"])):
        return 2
    return 1

def process_raw_input_data(base_x, base_y, step_table):
    processed_input = pd.DataFrame(index=step_table.index)

    processed_input["action"] = step_table["decided_action"].replace({
        "StepForward": 0,
        "GraspAnItem": 1,
        "TurnRight": 2,
        "TurnLeft": 3,
        "Destroy": 4,
        "StartFire": 5,
        "PutDownAnItem": 6,
        "DoNothing": 7,
    }).astype(int)

    processed_input["gps_difference"] = abs(step_table["measured_gps_x"] - step_table["real_gps_x"]) + abs(step_table["measured_gps_y"] - step_table["real_gps_y"])

    processed_input["orientation_difference"] = step_table.apply(compare_orientation, axis=1)

    processed_input["lidar_difference"] = abs(step_table["measured_lidar_distance"] - step_table["real_distance"])
    processed_input["radar_difference"] = abs(step_table["measured_radar_distance"] - step_table["real_distance"])
    processed_input["thermometer_difference"] = abs(step_table["measured_temperature"] - step_table["real_temperature"])

    processed_input["cardinality_base_distance"] = (abs(step_table["real_gps_x"] - base_x) + abs(step_table["real_gps_y"] - base_y)) / step_table["cardinality"]

    processed_input["action_successful"] = step_table.apply(lambda row: 1 if row["action_successful"] else 0, axis=1)

    return processed_input

In [ ]:
BASE_X = 7
BASE_Y = 12

processed_input = process_raw_input_data(BASE_X, BASE_Y, fact_step)

## Adding Trustor's Knowledge

In [ ]:
def add_trustor_knowledge(dataset, map_state_table, map_point_table, memory_map_table):
    real_map = map_state_table.join(map_point_table, on="map_point_id").reset_index()[["time", "x", "y", "terrain_type"]].set_index(["time", "x", "y"])

    idx = pd.IndexSlice
    def difference(memory):
        memory = memory.reset_index()
        different = memory[memory.apply(lambda row: real_map.at[(row["time"], row["x"], row["y"]), "terrain_type"] != row["terrain_type"], axis=1)]
        memory_overlap_coeff = 0.5 + 0.5 * memory["terrain_type"].count() / real_map.loc[idx[0, :, :]]["terrain_type"].count()
        memory_difference = different["terrain_type"].count() / memory["terrain_type"].count()

        return 1 - (memory_difference * memory_overlap_coeff)

    dataset["memory_similarity"] = memory_map_table.groupby(level=[0, 1]).apply(difference)

In [ ]:
add_trustor_knowledge(processed_input, fact_map_point_state, dym_map_point, fact_memory_map_point)
processed_input.to_csv("clean_training_data.csv")

## Machine Learning Model

In [ ]:
processed_input["profile"] = processed_input.apply(
    lambda row: dym_robot.at[row.name[1], "profile"], axis=1
).replace({
    "Normal": 0,
    "Broken": 1,
    "ItemDestroyer": 2,
    "Liar": 3,
    "Arsonist": 4
}).astype(int)

In [ ]:
processed_input = pd.read_csv("clean_training_data.csv", index_col=["robot_id", "start_time"])
past_experience = pd.DataFrame(index=processed_input.index)
past_experience["mission_experience"] = 0.5
past_experience["information_experience"] = 0.5
past_experience["behavior_experience"] = 0.5
past_experience["change_coeff"] = 1
past_experience.sort_index(inplace=True)

past_experience["change_coeff"] = 0.99 ** past_experience["change_coeff"].groupby(level=0).expanding().sum().values
all_raw = past_experience.join(processed_input).join(fact_step)

In [ ]:
def update_behavior_experience(row):
    if row["decided_action"] == "StartFire":
        return 0
    if row["decided_action"] == "PutDownAnItem":
        return 1
    return 0.5


def update_information_experience(row):
    return max(0, 1 - row["orientation_difference"] / 10 - row["gps_difference"] / 10 - row["lidar_difference"] / 20 - row["radar_difference"] / 20 - row["thermometer_difference"] / 10)


def update_mission_experience(row):
    return 0.5 if np.isnan(row["cardinality_base_distance"]) else max(0, 1 - max(0, row["cardinality_base_distance"] - 8) / 10)

past_experience["mission_experience"] = 0.3 + 0.7 * all_raw.apply(update_mission_experience, axis=1)
past_experience["information_experience"] = 0.3 + 0.7 * all_raw.apply(update_information_experience, axis=1)
past_experience["behavior_experience"] = 0.3 + 0.7 * all_raw.apply(update_behavior_experience, axis=1)

past_experience = past_experience.groupby(level=0).rolling(window=5, min_periods=1, closed="right").mean()
past_experience.reset_index(level=1, inplace=True)
past_experience.drop(columns=["robot_id"], inplace=True)

past_experience = past_experience.groupby(level=0).rolling(window=20, min_periods=1, closed="right").min()
past_experience.reset_index(level=1, inplace=True)
past_experience.drop(columns=["robot_id"], inplace=True)

past_experience = past_experience.groupby(level=0).ewm(alpha=2/3).mean()
past_experience.reset_index(level=1, inplace=True)
past_experience.drop(columns=["robot_id"], inplace=True)

past_experience

In [ ]:
train_experiences = past_experience.copy()
train_experiences

In [ ]:
past_experience = past_experience.groupby(level=0).shift(1).dropna()
past_experience

In [ ]:
df = (past_experience.join(processed_input)
      .join(train_experiences, lsuffix="_past")
      .reset_index()
      .drop(columns=["start_time", "robot_id", "change_coeff_past", "profile"]))

train_df = df.sample(frac=0.8, random_state=13)
test_df = df.drop(train_df.index)

train_X = train_df.copy()
test_X = test_df.copy()

train_Y = np.array(train_X[["mission_experience", "information_experience", "behavior_experience"]],
                   dtype="float32")
test_Y = np.array(test_X[["mission_experience", "information_experience", "behavior_experience"]],
                  dtype="float32")

train_X.drop(columns=["mission_experience", "information_experience", "behavior_experience"],
             inplace=True)
test_X.drop(columns=["mission_experience", "information_experience", "behavior_experience"],
            inplace=True)

train_X = np.array(train_X, dtype="float32")
test_X = np.array(test_X, dtype="float32")

In [ ]:
from sklearn.base import BaseEstimator
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score

class TrustVectorEstimator(BaseEstimator):
    def __init__(self, vector_size):
        super().__init__()
        self.vector_size = vector_size
        self.trees = [DecisionTreeRegressor(max_depth=5) for _ in range(vector_size)]

    def fit(self, X, Y):
        for i, tree in enumerate(self.trees):
            tree.fit(X, Y[:, i])

    def predict(self, X):
        return np.vstack([tree.predict(X) for tree in self.trees]).T

    def score(self, X, Y):
        predicted_Y = self.predict(X)
        return r2_score(Y, predicted_Y)

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

model = make_pipeline(StandardScaler(), TrustVectorEstimator(3))
model.fit(train_X, train_Y)

In [ ]:
from sklearn.metrics import mean_squared_error

predicted_Y = model.predict(test_X)
print(r2_score(test_Y, predicted_Y))
print(mean_squared_error(test_Y, predicted_Y))

In [ ]:
from sklearn.inspection import permutation_importance

dict(zip(df.columns, permutation_importance(model, test_X, test_Y).importances_mean))

## Indirect Trust Application

In [ ]:
peer_train_df = df.sample(frac=0.8, random_state=19)
peer_test_df = df.drop(peer_train_df.index)

peer_train_X = peer_train_df.copy()
peer_test_X = peer_test_df.copy()

peer_train_Y = np.array(peer_train_X[["mission_experience", "information_experience", "behavior_experience"]], dtype="float32")
peer_test_Y = np.array(peer_test_X[["mission_experience", "information_experience", "behavior_experience"]], dtype="float32")

peer_train_X.drop(columns=["mission_experience", "information_experience", "behavior_experience"], inplace=True)
peer_test_X.drop(columns=["mission_experience", "information_experience", "behavior_experience"], inplace=True)

peer_train_X = np.array(peer_train_X, dtype="float32")
peer_test_X = np.array(peer_test_X, dtype="float32")

In [ ]:
peer_model = make_pipeline(StandardScaler(), TrustVectorEstimator(3))
peer_model.fit(peer_train_X, peer_train_Y)

In [ ]:
peer_predicted_Y = model.predict(peer_test_X)
print(r2_score(peer_test_Y, peer_predicted_Y))
print(mean_squared_error(peer_test_Y, peer_predicted_Y))

## Pipeline Demonstration

In [ ]:
demo_fact_map_point_state = pd.read_csv("demo_data/fact_map_point_state.tsv", sep="\t", index_col=["time", "map_point_id"])
demo_fact_step = pd.read_csv("demo_data/fact_step.tsv", sep="\t", index_col=["start_time", "robot_id"])
demo_fact_memory_map_point = pd.read_csv("demo_data/fact_memory_map_point.tsv", sep="\t", index_col=["time", "robot_id", "x", "y"])
demo_dym_map_point = pd.read_csv("demo_data/dym_map_point.tsv", sep="\t", index_col=["id"])
demo_dym_robot = pd.read_csv("demo_data/dym_robot.tsv", sep="\t", index_col=["id"])

In [ ]:
DEMO_BASE_X = 21
DEMO_BASE_Y = 28

demo_processed_input = process_raw_input_data(DEMO_BASE_X, DEMO_BASE_Y, demo_fact_step)
add_trustor_knowledge(demo_processed_input, demo_fact_map_point_state, demo_dym_map_point, demo_fact_memory_map_point)
demo_processed_input.to_csv("clean_demo_data.csv")

In [ ]:
demo_past_experience = {robot_id: {
    "mission_experience": 0.5,
    "information_experience": 0.5,
    "behavior_experience": 0.5
    } for robot_id in demo_processed_input.index.get_level_values(1).unique()
}

DEMO_PEER_OPINION = 0.6

for (timestamp, robot_id), step in demo_processed_input.iterrows():
    model_input = np.hstack([step, list(demo_past_experience[robot_id].values())]).reshape(1, -1)
    trust_vector = peer_model.predict(model_input)[0]

    peer_opinion = peer_model.predict(model_input)[0]
    trust_vector = [v * (DEMO_PEER_OPINION * (p - 1) + 2) / 2 for v, p in zip(trust_vector, peer_opinion)]

    demo_past_experience[robot_id]["mission_experience"] = trust_vector[0]
    demo_past_experience[robot_id]["information_experience"] = trust_vector[1]
    demo_past_experience[robot_id]["behavior_experience"] = trust_vector[2]

MISSION_THRESHOLD = 0.8
INFORMATION_THRESHOLD = 0.6
BEHAVIOR_THRESHOLD = 0.45

print("Trust decision:")
print("ID   MISSION   INFORMATION   BEHAVIOR")

for robot_id, trust_vector in sorted(demo_past_experience.items()):
    print(f"{robot_id:2}   ", end="")
    print("  YES  " if trust_vector["mission_experience"] > MISSION_THRESHOLD else "  NO   ", end="   ")
    print("    YES    " if trust_vector["information_experience"] > INFORMATION_THRESHOLD else "    NO     ", end="   ")
    print("  YES   " if trust_vector["behavior_experience"] > BEHAVIOR_THRESHOLD else "   NO   ")